In [5]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
shelter = AnimalShelter()

# class read method must support return of cursor object
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash("BradJacksonProjectTwo")

image_filename = "logo.png"
encoded_image = base64.b64encode(open(image_filename, "rb").read())


app.layout = html.Div(
    [
        html.Center(
            html.Img(
                src=f"data:image/png;base64,{encoded_image.decode()}",
                style={"width": "400px"},
            )
        ),
        html.Center(html.B(html.H1("Brad Jackson's SNHU CS-340 Dashboard"))),
        html.Hr(),
        html.Div(
            [
                dcc.Dropdown(
                    id="filter-type",
                    options=[
                        {"label": "Water Rescue", "value": "0"},
                        {"label": "Mountain or Wilderness Rescue", "value": "1"},
                        {
                            "label": "Disaster or Individual Tracking",
                            "value": "2",
                        },
                        {"label": "Reset", "value": "3"},
                    ],
                    value="3",
                ),
            ]
        ),
        html.Hr(),
        dt.DataTable(
            id="datatable-id",
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True}
                for i in df.columns
            ],
            data=df.to_dict("records"),
            sort_action="native",
            page_action="native",
            column_selectable=False,
            sort_mode="multi",
            selected_columns=[],
            page_current=0,
            page_size=10,
            selected_rows=[0],
            row_selectable=False,
            row_deletable=False,
            editable=False,
        ),
        html.Br(),
        html.Hr(),
        # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
        html.Div(
            className="row",
            style={"display": "flex"},
            children=[
                html.Div(
                    id="graph-id",
                    className="col s12 m6",
                ),
                html.Div(
                    id="map-id",
                    className="col s12 m6",
                ),
            ],
        ),
    ]
)

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(
    [Output("datatable-id", "data"),Output("datatable-id", "columns")],
    [Input("filter-type", "value")],
)
def update_dashboard(filter_type):
    # below runs the queries for each filter type

    cur = None
    if filter_type == "0":
        cur = shelter.read(
            {
                "animal_type": "Dog",
                "breed": {
                    "$in": [
                        "Labrador Retriever Mix",
                        "Chesapeake Bay Retriever",
                        "Newfoundland",
                    ]
                },
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte": 26},
                "age_upon_outcome_in_weeks": {"$lte": 156},
            }
        )

    elif filter_type == "1":
        cur = shelter.read(
            {
                "animal_type": "Dog",
                "breed": {
                    "$in": [
                        "German Shephard",
                        "Alaskan Malamute",
                        "Old English Sheepdog",
                        "Siberian Husky",
                        "Rottweiler",
                    ]
                },
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 26},
                "age_upon_outcome_in_weeks": {"$lte": 156},
            }
        )

    elif filter_type == "2":
        cur = shelter.read(
            {
                "animal_type": "Dog",
                "breed": {
                    "$in": [
                        "Doberman Pinscher",
                        "German Shepherd",
                        "Golden Retriever",
                        "Bloodhound",
                        "Rottweiler",
                    ]
                },
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte": 20},
                "age_upon_outcome_in_weeks": {"$lte": 300},
            }
        )
    else:
        cur = shelter.read({})  # if reset we just get the whole list

    df = pd.DataFrame.from_records(cur)
    columns = [
        {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
    ]
    data = df.to_dict("records")
    return (data,columns)


@app.callback(
    Output("datatable-id", "style_data_conditional"),
    [Input("datatable-id", "selected_columns")],
)
def update_styles(selected_columns):
    return [
        {"if": {"column_id": i}, "background_color": "#D2F3FF"}
        for i in selected_columns
    ]


@app.callback(
    Output("graph-id", "children"), [Input("datatable-id", "derived_viewport_data")]
)
def update_graphs(viewData):
    # Gathers dog breed and count to display on a pie chart for data currently showing on table above
    dff = pd.DataFrame.from_dict(viewData)
    k = dff["breed"].value_counts().keys().tolist()
    v = dff["breed"].value_counts().tolist()
    return [
        dcc.Graph(
            figure=px.pie(
                data_frame=dff,
                names=k,
                values=v,
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=725,
                height=500,
            )
        )
    ]


@app.callback(
    Output("map-id", "children"), [Input("datatable-id", "derived_viewport_data")]
)
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(
            style={"width": "1000px", "height": "500px"},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[30.75, -97.48],
                    children=[
                        dl.Tooltip(dff.iloc[0, 4]),
                        dl.Popup(
                            [
                                html.H1("ID"),
                                html.P(dff.iloc[0, 2]),
                                html.H3(dff.iloc[0, 9]),
                                html.H3("Type"),
                                html.P(dff.iloc[0, 3]),
                                html.H3("Color"),
                                html.P(dff.iloc[0, 5]),
                            ]
                        ),
                    ],
                ),
            ],
        )
    ]


app
